In [1]:
import  numpy as np
import pandas as pd
import ast
import nltk
import pickle as pk

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
credits.rename(columns = {'movie_id' : 'id', 'title' : 'title', 'cast' : 'cast', 'crew' : 'crew'}, inplace=True)

In [4]:
movies = movies.merge(credits, on='id')

In [5]:
movies = movies[['id','original_title','genres','keywords','overview','production_companies','cast','crew']]

In [6]:
def conv(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [7]:
movies['genres'] = movies['genres'].apply(conv)

In [8]:
movies['keywords'] = movies['keywords'].apply(conv)

In [9]:
movies['production_companies'] = movies['production_companies'].apply(conv)

In [10]:
def convert(obj):
    L=[]
    c=0
    for i in ast.literal_eval(obj):
        if c!=5:
            L.append(i['name'])
            c+=1
        else:
            break
    return L

In [11]:
movies['cast'] = movies['cast'].apply(convert)

In [12]:
def dp(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L

In [13]:
movies['crew'] = movies['crew'].apply(dp)

In [14]:
movies['overview'] = movies['overview'].apply(lambda x:str(x).split())

In [15]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['production_companies'] = movies['production_companies'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [16]:
movies["tag"] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['production_companies'] + movies['cast'] + movies['crew']

In [17]:
new_df = movies[['id','original_title','tag']]

In [18]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [19]:
new_df['tag'] = new_df['tag'].apply(lambda x:[ps.stem(i) for i in x])

C:\Users\ayan2\AppData\Local\Temp\ipykernel_20376\2338132760.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tag'] = new_df['tag'].apply(lambda x:[ps.stem(i) for i in x])


In [20]:
new_df['tag'] = new_df['tag'].apply(lambda x:' '.join(x))

C:\Users\ayan2\AppData\Local\Temp\ipykernel_20376\2297916504.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tag'] = new_df['tag'].apply(lambda x:' '.join(x))


In [21]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=7000, stop_words='english')

In [22]:
vec = cv.fit_transform(new_df['tag']).toarray()

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
sim = cosine_similarity(vec)

In [25]:
def recommend(mov,count):
    a = 6 + count*5
    mov_index = new_df[new_df['original_title'] == mov].index[0]
    mov_list = sorted(list(enumerate(sim[mov_index])),reverse=True,key=lambda x:x[1])[1:a]
    
    for i in mov_list:
        print(new_df['original_title'][i[0]])

In [26]:
recommend("Spider-Man",0)

Spider-Man 3
Spider-Man 2
The Amazing Spider-Man 2
Arachnophobia
The Amazing Spider-Man


In [ ]:
pk.dump()